This script illustrate the Multi-Scale Cross-Matching pre-training

In [ ]:
from mydataset import MyDataset_MultiScale_PreTrain
import myaugmentation as MyAug
import mymodels
import myloss

import os 
import tqdm
import numpy as np
import matplotlib.pyplot as plt

import torch 
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts

from torch.cuda.amp import GradScaler, autocast
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import torch.nn.parallel as parallel

In [ ]:
class MLP(nn.Module):
    def __init__(self, input_size=2048, hidden_size=512, output_size=2):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [ ]:
if torch.cuda.is_available():
    num_gpus = torch.cuda.device_count()
    print(f"Number of available GPUs: {num_gpus}")

    # Get the names of the available GPUs
    gpu_names = [torch.cuda.get_device_name(i) for i in range(num_gpus)]
    print("Available GPUs:")
    for i, gpu_name in enumerate(gpu_names):
        print(f"GPU {i}: {gpu_name}")
else:
    print("CUDA is not available on this system.")       

In [ ]:
# Set Hyperparameters
epochs = 25
batch_size = 56
num_workers = 2
learning_rate = 1e-3
optimizer_name = "AdamW"

output_path = "."
model_name = "ResNet50_Pre-Train"
metadataTrain = "./metadata/train.csv" # list of training files

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
writer = SummaryWriter(log_dir=os.path.join(output_path, model_name, "log", "pre-training"))

# Create Dataset and Dataloader
dataset = MyDataset_MultiScale_PreTrain(basic_transform=MyAug.base_aug(), 
                                        aug_transform=MyAug.pretrain_aug(),
                                        metadata=metadataTrain)

dataloader = DataLoader(dataset, batch_size=batch_size, 
                        num_workers=num_workers, shuffle=True)

In [ ]:
# Create model and prepare for training
model = mymodels.ResNet50_with_Feature()
model = parallel.DataParallel(model)
model.to(device)

criterion_infonce = myloss.InfoNCE()
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2)
scaler = GradScaler() # the scaler for mixed precision training

In [ ]:
# Train the model
for epoch in tqdm.tqdm(range(epochs)):
    
    model.train()
    running_loss = 0.0
    # for accuracy
    total = 0
    correct = 0 # w/o augmentation
        
    for i, batch in tqdm.tqdm(enumerate(dataloader)):
        # Move the data to the device
        image_1 = batch[1].to(device) # the image patch
        image_2 = batch[2].to(device) # the whole image w/ mask
        
        # Zero the gradients
        optimizer.zero_grad()
        
        with autocast():
            _, image_embeddings = model(image_1)
            _, patch_embeddings = model(image_2)
            
            # InfoNCE Loss
            loss = criterion_infonce(image_embeddings, patch_embeddings                                                

        # Perform the backward pass
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # Update the learning rate
        scheduler.step()

        # Update the running loss
        running_loss += loss.item() * batch_size

        # Log training loss to TensorBoard
        writer.add_scalar('Running/Loss', loss.item(), epoch*len(dataloader)+i)        
        writer.add_scalar('Running/Loss-InfoNCE', loss_infonce.item(), epoch*len(dataloader)+i)
        writer.add_scalar('Running/LR', optimizer.param_groups[0]['lr'], epoch*len(dataloader)+i)   
        
        
    # Calculate the average loss
    epoch_loss = running_loss / len(dataloader.dataset)
    writer.add_scalar('Epoch/Loss', epoch_loss, epoch)

    if use_classifier:
        accuracy_overall = correct / total
        writer.add_scalar('Epoch/Acc', accuracy_overall, epoch)
    # Save the model
    torch.save(model.state_dict(), os.path.join(output_path, model_name+f'_{epoch}_{epoch_loss}.pth'))
